# Práctica Pipelines - Dataset titanic

En el siguiente dataset intentaremos clasificar supervivientes del titanic en función de sus variables.

# Dataset "Titanic"


In [44]:
import pandas as pd

In [45]:
df = pd.read_csv("https://raw.githubusercontent.com/PabloSoligo2014/3670-UNLaM-CD/refs/heads/main/datasets/titanic.csv")

In [46]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
df.isnull().sum()/len(df)*100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

# Parte 1

Proceso el dataset; aplicando pipelines quite las columnas que considere, realice las imputaciones y/o escalamientos según corresponda y su conocimiento e intuicion le indiquen. Intente clasificar usando KNN.


In [48]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier

In [49]:
df = pd.read_csv("https://raw.githubusercontent.com/PabloSoligo2014/3670-UNLaM-CD/refs/heads/main/datasets/titanic.csv")

In [50]:
X = df.drop(["PassengerId", "Survived", "Cabin", "Name", "Ticket"], axis=1)
y = df["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3, test_size=0.3)

Campos filtrados

In [51]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
233,3,female,5.0,4,2,31.3875,S
229,3,female,NaN,3,1,25.4667,S
356,1,female,22.0,0,1,55.0000,S
439,2,male,31.0,0,0,10.5000,S
637,2,male,31.0,1,1,26.2500,S
...,...,...,...,...,...,...,...
256,1,female,NaN,0,0,79.2000,C
643,3,male,NaN,0,0,56.4958,S
249,2,male,54.0,1,0,26.0000,S
664,3,male,20.0,1,0,7.9250,S


## No funciona por separado (Revisar)

In [52]:
ct = ColumnTransformer([("mean_Age", SimpleImputer(strategy="mean"), ["Age"]),
                        ("most_freq_cat", SimpleImputer(strategy="most_frequent") , ["Embarked"])
                        ])

In [53]:
ct2 = ColumnTransformer([("OneHot", OneHotEncoder(sparse_output=False, dtype=int, drop='first') , [1])],
                         remainder='passthrough')

In [54]:
ct3 = ColumnTransformer([("OneHot", OneHotEncoder(sparse_output=False, dtype=int, drop='first') , [6])],
                         remainder='passthrough')

In [55]:
pl = Pipeline(steps=[
    ("ct", ct),
    ("ct2", ct2),
    ("ct3", ct3),
    ("est", KNeighborsClassifier())
])

## Solucion (Es valido por el profe)

In [56]:
ct = ColumnTransformer([
    ("num", Pipeline([
        ("imp", SimpleImputer(strategy="mean")),
        ("sc", StandardScaler())
    ]), [0, 2, 5]),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("oh", OneHotEncoder(drop='first'))
    ]), [1, 6])
])

In [57]:
pl = Pipeline(steps=[
    ("ct", ct),
    ("est", KNeighborsClassifier())
])

In [58]:
pl.fit(X_train, y_train)
y_pred = pl.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.77      0.80       164
           1       0.68      0.76      0.72       104

    accuracy                           0.77       268
   macro avg       0.76      0.77      0.76       268
weighted avg       0.78      0.77      0.77       268



# Parte 2

Reutilizando lo anterior tanto como sea posible evalue utilizando otro clasificador. Pruebe distintas alternativas de transformaciones. Evalue que opción (transformaciones+clasificador) ofrece los mejores resultados.

In [59]:
ct4 = ColumnTransformer(
    transformers=[
      ("onehot", OneHotEncoder(drop='first'), [1, 2]),    # columna categórica
      ("scaler", StandardScaler(), [0, 2]),               # columnas numéricas
    ]
)

In [60]:
pl = Pipeline(steps=[("ct", ct),
                     ("ct4", ct4),
                     ("est", KNeighborsClassifier())])

## Arbol de decision

In [61]:
ct = ColumnTransformer([
    ("num", Pipeline([
        ("imp", SimpleImputer(strategy="mean")),
        ("sc", StandardScaler())
    ]), [0, 2, 5]),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("oh", OneHotEncoder(drop='first'))
    ]), [1, 6])
])

In [62]:
pl = Pipeline(steps=[
    ("ct", ct),
    ("est", DecisionTreeClassifier())
])

In [63]:
pl.fit(X_train, y_train)
y_pred = pl.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.81      0.81       164
           1       0.70      0.68      0.69       104

    accuracy                           0.76       268
   macro avg       0.75      0.75      0.75       268
weighted avg       0.76      0.76      0.76       268



## Random Forest

In [64]:
ct = ColumnTransformer([
    ("num", Pipeline([
        ("imp", SimpleImputer(strategy="mean")),
        ("sc", StandardScaler())
    ]), [0, 2, 5]),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("oh", OneHotEncoder(drop='first'))
    ]), [1, 6])
])

In [65]:
pl = Pipeline(steps=[
    ("ct", ct),
    ("est", RandomForestClassifier())
])

In [66]:
pl.fit(X_train, y_train)
y_pred = pl.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.82      0.83       164
           1       0.73      0.74      0.73       104

    accuracy                           0.79       268
   macro avg       0.78      0.78      0.78       268
weighted avg       0.79      0.79      0.79       268



## Naive Bayes

In [67]:
ct = ColumnTransformer([
    ("num", Pipeline([
        ("imp", SimpleImputer(strategy="mean")),
        ("sc", StandardScaler())
    ]), [0, 2, 5]),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("oh", OneHotEncoder(drop='first'))
    ]), [1, 6])
])

In [68]:
pl = Pipeline(steps=[
    ("ct", ct),
    ("est", GaussianNB())
])

In [69]:
pl.fit(X_train, y_train)
y_pred = pl.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.82      0.81       164
           1       0.71      0.69      0.70       104

    accuracy                           0.77       268
   macro avg       0.76      0.75      0.76       268
weighted avg       0.77      0.77      0.77       268

